In [1]:
%pylab inline
rcParams["figure.figsize"] = (16,5)
from scipy.io import wavfile
import pyworld
import pysptk
import IPython
from IPython.display import Audio
import librosa
import librosa.display
from glob import glob

Populating the interactive namespace from numpy and matplotlib


In [129]:
from nnmnkwii.preprocessing import trim_zeros_frames, remove_zeros_frames
import re
from nnmnkwii.io import hts

In [ ]:
from scipy import stats

In [26]:
import pandas as pd

In [2]:
import scipy

In [21]:
import numpy as np

In [3]:
import os
import re

In [51]:
lab_dir="/home/patrick/corpous/name_clean3/label_phone_align"
ori_dir="/home/patrick/corpous/name_clean3"
target_dir="/home/patrick/corpous/name_clean3/s100/remove_silence"
csv_file="/home/patrick/corpous/name_clean3/stats.csv"

In [11]:
norm_dir="/home/patrick/corpous/name_clean4"

In [60]:
story_map = {"s100":0, "s101":1,"s102":2,"s103":3,"s104":4,"s26":5,"s27":6,"s29":7,"s30":8,"s31":9,"s57":10,"s58":11,"s59":12,"s105":13}
for s_key in story_map.keys():
    total_duration = 0
    total_utterance = 0
    max_value = 0
    min_value = 0
    for wav_path in glob(ori_dir+"/*.wav"):
        basename=os.path.basename(wav_path)
        utt_id = basename.replace('.wav','')
        s_index = re.split('_',utt_id)[0]
        if s_key != s_index:
            continue
        x, fs = librosa.core.load(wav_path,sr=16000)
        if np.max(x) > max_value:
            max_value = np.max(x)
        if np.min(x) < min_value:
            min_value = np.min(x)
    for wav_path in glob(ori_dir+"/*.wav"):
        basename=os.path.basename(wav_path)
        utt_id = basename.replace('.wav','')
        s_index = re.split('_',utt_id)[0]
        if s_key != s_index:
            continue
        x, fs = librosa.core.load(wav_path,sr=16000)
        x2 = ((x-min_value)/(max_value-min_value)) - 0.5
        x2 = x2*32768
        x2=x2.astype(np.int16)
        scipy.io.wavfile.write(os.path.join(norm_dir, basename),fs, x2)
    print("story index {0}, max value {1},min value {2}".format(s_key, max_value, min_value))

story index s100, max value 0.529083251953125,min value -0.52044677734375
story index s101, max value 0.34722900390625,min value -0.28070068359375
story index s102, max value 0.999969482421875,min value -1.0
story index s103, max value 0.288238525390625,min value -0.294189453125
story index s104, max value 0.999969482421875,min value -0.999969482421875
story index s26, max value 0.999969482421875,min value -1.0
story index s27, max value 0.999969482421875,min value -1.0
story index s29, max value 0.999969482421875,min value -1.0
story index s30, max value 0.295684814453125,min value -0.36639404296875
story index s31, max value 0.999969482421875,min value -0.777679443359375
story index s57, max value 0.406768798828125,min value -0.4609375
story index s58, max value 0.311553955078125,min value -0.364166259765625
story index s59, max value 0.32672119140625,min value -0.37591552734375
story index s105, max value 0.33685302734375,min value -0.3370361328125


In [12]:
test_wav_s101="/home/patrick/corpous/name_clean3/s26_out_0.wav"

In [115]:
y, fs = librosa.core.load(test_wav_s101,sr=16000)

In [116]:
f0, timeaxis = pyworld.harvest(y.astype(np.float64), fs, frame_period=5, f0_floor=71.0, f0_ceil=700)
spectrogram = pyworld.cheaptrick(y.astype(np.float64), f0, timeaxis, fs)
mgc = pysptk.sp2mc(spectrogram, 59, alpha=pysptk.util.mcepalpha(fs))

In [121]:
spectrogram.shape

(2333, 513)

In [123]:
a=np.sum(spectrogram,axis=1)

In [127]:
a[100:200]

array([  5.37876300e-05,   8.49321807e-05,   2.49986315e-03,
         2.62847910e-01,   3.47910937e-01,   4.76633762e-01,
         5.84394769e-01,   7.82466184e-01,   8.67154767e-01,
         1.09742125e+00,   2.14258024e+00,   2.75249008e+00,
         4.23170085e+00,   5.45022411e+00,   6.64372976e+00,
         7.20350794e+00,   7.82697631e+00,   7.19550109e+00,
         6.19134054e+00,   6.20063553e+00,   6.08067318e+00,
         5.33881638e+00,   3.79378088e+00,   2.96855350e+00,
         2.22909184e+00,   1.86641785e+00,   1.72159351e+00,
         1.68899468e+00,   1.67400509e+00,   1.83498114e+00,
         1.74965419e+00,   1.69832666e+00,   1.64571438e+00,
         1.65624196e+00,   1.57530603e+00,   1.59591025e+00,
         1.68542745e+00,   1.56420331e+00,   1.33292732e+00,
         1.36481093e+00,   1.38743073e+00,   1.31403583e+00,
         1.32474770e+00,   1.21804459e+00,   1.14838366e+00,
         1.07690860e+00,   6.85797310e-01,   3.98365864e-01,
         2.44480041e-01,

In [77]:
np.mean(mgc[:,0])

-6.3847275409131168

In [15]:
Audio(y,rate=16000)

In [114]:
f0, timeaxis = pyworld.harvest(y.astype(np.float64), fs, frame_period=5, f0_floor=71.0, f0_ceil=700)
spectrogram = pyworld.cheaptrick(y, f0, timeaxis, fs)

ValueError: Buffer dtype mismatch, expected 'double' but got 'float'

In [16]:
rmse=librosa.feature.rmse(y.astype(np.float64), frame_length=400,hop_length=80)

In [19]:
rmse[0,:]

array([ 0.00143944,  0.00146358,  0.00161668, ...,  0.00032712,
        0.00027816,  0.00028225], dtype=float32)

In [50]:
np.max(rmse[0,:])

0.22767136

In [62]:
df = pd.DataFrame(rmse[0,:])
df.quantile(.99)

0    0.073936
Name: 0.99, dtype: float64

In [108]:
lab_dir="/home/patrick/corpous/name_clean3/label_phone_align"

In [157]:
s_key="s104"
mean_list=[]
mean_0_mc=[]
for wav_path in glob(ori_dir+"/*.wav"):
    basename=os.path.basename(wav_path)
    label_path = os.path.join(lab_dir, basename.replace('wav','lab'))
    utt_id = basename.replace('.wav','')
    s_index = re.split('_',utt_id)[0]
    if s_key != s_index:
        continue
    y, fs = librosa.core.load(wav_path,sr=16000)
    labels = hts.load(label_path)
    silence_indices = labels.silence_frame_indices()
    f0, timeaxis = pyworld.harvest(y.astype(np.float64), fs, frame_period=5, f0_floor=71.0, f0_ceil=700)
    spectrogram = pyworld.cheaptrick(y.astype(np.float64), f0, timeaxis, fs)
    silence_removed_spectrogram = np.delete(spectrogram[:labels.num_frames()], silence_indices, axis=0)
    mean = np.mean(np.sum(silence_removed_spectrogram,axis=1))
#     mgc = pysptk.sp2mc(spectrogram, 59, alpha=pysptk.util.mcepalpha(fs))
#     mean_zero_mc = np.mean(mgc[:,0])
#     mean_0_mc.append(mean_zero_mc)
#     rmse=librosa.feature.rmse(y, frame_length=400,hop_length=80)
#     mean=np.mean(rmse[0,:])
    mean_list.append(mean)
#     df = pd.DataFrame(rmse[0,:])
#     max_v = df.quantile(.95)
#     min_v = df.quantile(0.05)
    print("story index {0}, mean value {1}".format(basename, mean))

story index s104_73.wav, mean value 2.2968432676544963
story index s104_93.wav, mean value 2.665460384199308
story index s104_27.wav, mean value 1.7047393321134807
story index s104_74.wav, mean value 1.8942444269348417
story index s104_88.wav, mean value 1.5915530304117889
story index s104_29.wav, mean value 2.6498162322521788
story index s104_83.wav, mean value 1.6182218641506712
story index s104_82.wav, mean value 0.9345119729548791
story index s104_11.wav, mean value 1.6376016407552765
story index s104_6.wav, mean value 1.4140878468472546
story index s104_17.wav, mean value 2.807982064537738
story index s104_37.wav, mean value 2.1030222158238985
story index s104_28.wav, mean value 1.0506344737153852
story index s104_87.wav, mean value 0.9700498101064282
story index s104_4.wav, mean value 1.5526562079634654
story index s104_97.wav, mean value 1.4250169132892314
story index s104_67.wav, mean value 3.945139512616123
story index s104_72.wav, mean value 3.710783948315497
story index s104

In [132]:
boston_df = pd.Series(mean_list)


In [158]:
z = np.abs(stats.zscore(mean_list))
print(z)

[ 0.02148854  0.22417409  0.4160926   0.28979808  0.49152492  0.21374814
  0.47375164  0.92940592  0.4608361   0.60979551  0.31915679  0.1506594
  0.85201679  0.90572195  0.51744747  0.6025119   1.07700844  0.9208236
  0.34214526  0.19006761  0.20985711  0.66636583  0.62991582  0.83660534
  0.31341152  0.06562198  0.39668391  2.30817729  0.83835817  1.10049504
  0.57535821  3.64387775  0.57767179  0.60270846  0.67182381  0.25664406
  0.65456092  2.8510439   0.09812934  0.3939208   0.49303621  0.47997638
  0.3528996   0.43758605  0.7204528   0.23936116  0.57961905  0.5124465
  0.67342718  0.06590797  0.39526424  0.12811415  0.03564397  0.08177947
  0.13159743  2.92068431  0.15388072  1.67919177  0.06198575  0.67024747
  0.51323589  0.51436615  0.19904596  0.20715096  0.33449121  1.08255408
  0.05835125  0.22872533  0.27682819  0.10894381  0.55502843  0.91964795
  0.89164269  0.10241124  0.05809875  0.31914618  0.05263446  0.44724391
  0.69153036  0.65033747  0.52760166  0.04448979  0.20

In [159]:
threshold = 2.0
print(np.where(z > threshold))

(array([ 27,  31,  37,  55,  93, 102]),)


In [160]:
np.mean(mean_list)

2.3290868527117539

In [145]:
np.std(mean_list)

0.45613880461400441

In [155]:
np.std(mean_list)

0.29757780466981126

In [144]:
np.mean(s102_mean_list)

3.973861152538686

In [154]:
np.mean(s103_mean_list)

1.4629484369474577

In [146]:
np.std(s102_mean_list)

2.8317952785215841

In [138]:
s102_mean_list = mean_list

In [147]:
s103_mean_list = mean_list

In [174]:
s_key="cleverrabbitandthelion"
mean_list=[]
mean_0_mc=[]
ori_dir='/home/patrick/corpous/wav'
for wav_path in glob(ori_dir+"/*.wav"):
    basename=os.path.basename(wav_path)
    label_path = os.path.join(lab_dir, basename.replace('wav','lab'))
    utt_id = basename.replace('.wav','')
    s_index = re.split('_',utt_id)[4]
    if s_key != s_index:
        continue
    y, fs = librosa.core.load(wav_path,sr=16000)
#     labels = hts.load(label_path)
#     silence_indices = labels.silence_frame_indices()
    f0, timeaxis = pyworld.harvest(y.astype(np.float64), fs, frame_period=5, f0_floor=71.0, f0_ceil=700)
    spectrogram = pyworld.cheaptrick(y.astype(np.float64), f0, timeaxis, fs)
#     silence_removed_spectrogram = np.delete(spectrogram[:labels.num_frames()], silence_indices, axis=0)
    mean = np.mean(np.sum(spectrogram,axis=1))
#     mgc = pysptk.sp2mc(spectrogram, 59, alpha=pysptk.util.mcepalpha(fs))
#     mean_zero_mc = np.mean(mgc[:,0])
#     mean_0_mc.append(mean_zero_mc)
#     rmse=librosa.feature.rmse(y, frame_length=400,hop_length=80)
#     mean=np.mean(rmse[0,:])
    mean_list.append(mean)
#     df = pd.DataFrame(rmse[0,:])
#     max_v = df.quantile(.95)
#     min_v = df.quantile(0.05)
    print("story index {0}, mean value {1}".format(basename, mean))

story index blizzard2018_en_benchmark_fls_cleverrabbitandthelion_012_005.wav, mean value 3.5797152135406143
story index blizzard2018_en_benchmark_fls_cleverrabbitandthelion_003_001.wav, mean value 12.45521521192051
story index blizzard2018_en_benchmark_fls_cleverrabbitandthelion_001_002.wav, mean value 1.5936247885892665
story index blizzard2018_en_benchmark_fls_cleverrabbitandthelion_012_003.wav, mean value 1.210495089013438
story index blizzard2018_en_benchmark_fls_cleverrabbitandthelion_004_003.wav, mean value 1.3212112127127384
story index blizzard2018_en_benchmark_fls_cleverrabbitandthelion_007_002.wav, mean value 5.425733331252797
story index blizzard2018_en_benchmark_fls_cleverrabbitandthelion_008_001.wav, mean value 1.965251817694128
story index blizzard2018_en_benchmark_fls_cleverrabbitandthelion_004_004.wav, mean value 0.9062741741815373
story index blizzard2018_en_benchmark_fls_cleverrabbitandthelion_001_001.wav, mean value 1.339526572924129
story index blizzard2018_en_bench

In [175]:
np.mean(mean_list)

3.0995174877184661